In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 8.6 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=6a7fe51b9bd99dc2bbe4e71bb458e7d18ba125c30707729e3353700178280341
  Stored in directory: /root/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtools


In [ ]:
!pip install Transformers

In [ ]:
!pip install datasets

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-ae6b39ac68ed>", line 2, in <cell line: 2>
    wandb.login()
  File "/usr/local/lib/python3.9/dist-packages/wandb/sdk/wandb_login.py", line 77, in login
    configured = _login(**kwargs)
  File "/usr/local/lib/python3.9/dist-packages/wandb/sdk/wandb_login.py", line 298, in _login
    wlogin.prompt_api_key()
  File "/usr/local/lib/python3.9/dist-packages/wandb/sdk/wandb_login.py", line 221, in prompt_api_key
    key, status = self._prompt_api_key()
  File "/usr/local/lib/python3.9/dist-packages/wandb/sdk/wandb_login.py", line 201, in _prompt_api_key
    key = apikey.prompt_api_key(
  File "/usr/local/lib/python3.9/dist-packages/wandb/sdk/lib/apikey.py", line 132, in prompt_api_key
    key = input_callback(api_ask).strip()
  File "/usr/local/lib/python3.9/dist-packages/clic

TypeError: ignored

# a) 

In [ ]:

from datasets import load_dataset

data = load_dataset("wmt16", "de-en")

Found cached dataset wmt16 (C:/Users/taran/.cache/huggingface/datasets/wmt16/de-en/1.0.0/746749a11d25c02058042da7502d973ff410e73457f3d305fc1177dc0e8c4227)


  0%|          | 0/3 [00:00<?, ?it/s]

Taking 0.1% of the train dataset 

In [ ]:
data_train=data['train'].train_test_split(test_size=0.99)

In [ ]:
print(data_train)

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 45488
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 4503397
    })
})


In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
!pip install SentencePiece

In [ ]:
!pip install sacremoses

loading a Trikotman/GermantoHunsrikV1 tokenizer to process the German-English language pairs and then using the tokenizer to train the model.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Tritkoman/GermantoHunsrikV1")
model = AutoModelForSeq2SeqLM.from_pretrained("Tritkoman/GermantoHunsrikV1")



Tokenize the source (English) and target (German) separately because you can’t tokenize German text with a tokenizer pretrained on an English vocabulary.
Truncate sequences to be no longer than the maximum length set by the max_length parameter.

In [ ]:
source_lang = "en"
target_lang = "de"

def preprocess_function(examples):
    inputs = [example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=512, truncation=True)
    return model_inputs

In [ ]:
model

Output layer of the decoder layer is set to trainable 

In [ ]:
model.decoder.block[-1].layer.require_grad=True

In [ ]:
print(data_train['train'])

Tokenizing the train and validation set

In [ ]:
tokenized_data = data_train['train'].map(preprocess_function, batched=True)

In [ ]:
tokenized_val = data['validation'].map(preprocess_function, batched=True)

In [ ]:
!pip install evaluate

In [ ]:
!pip install sacrebleu

In [ ]:
import evaluate

metric = evaluate.load("sacrebleu")

In [ ]:
import numpy as np
from datasets import load_dataset, load_metric

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

#rouge_metric = load_metric('rouge')
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    #rouge_scores = rouge_metric.compute(predictions=generated_texts, references=reference_texts, rouge_types=['rouge-l'])['rouge-l']['fmeasure']
    result = {"bleu": result["score"]}
    #rouge_scores={"rouge": rouge_scores['rouge-l']}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

Using datacollector for dynamically padding the it to the max length text instead of padding all to the max length.

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="Tritkoman/GermantoHunsrikV1")

Training on 2 epochs with batch size 2

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="translation",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    push_to_hub=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Currently logged in as: saloni22062 (iiit-delhi). Use `wandb login --relogin` to force relogin


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,3.687900,2.606264,6.694900,38.127200
2,3.529100,2.529386,7.580500,36.504400


TrainOutput(global_step=45488, training_loss=3.9180581934469667, metrics={'train_runtime': 12450.5402, 'train_samples_per_second': 7.307, 'train_steps_per_second': 3.653, 'total_flos': 4331428940218368.0, 'train_loss': 3.9180581934469667, 'epoch': 2.0})

Saving the model

In [ ]:
trainer.save_model("/content/sample_data/Models")

In [ ]:
from google.colab import files
files.download("/content/sample_data/Models/pytorch_model.bin")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp -av '/content/sample_data/Models/pytorch_model.bin'  '/content/drive/MyDrive'

'/content/sample_data/Models/pytorch_model.bin' -> '/content/drive/MyDrive/pytorch_model.bin'


In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("saved_model")

# **C) Evaluate Scores**

In [ ]:
from datasets import load_dataset

test_dataset=data['test']


#lists initalization
generated_texts = []
reference_texts = []


for example in test_dataset['translation']:
    input_ids = tokenizer.encode(example['en'], truncation=True, padding=True, return_tensors='pt')
    output_ids = model.generate(input_ids, max_length=128, num_beams=4, early_stopping=True)
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # adding in the lists
    generated_texts.append(generated_text)
    reference_texts.append(example['de'])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24936 sha256=fabd39b22ba3ae4fdfcfb92b25198901e2f54b6e676e14bdfd3be3669c10ee89
  Stored in directory: c:\users\taran\appdata\local\pip\cache\wheels\5a\a8\47\66a086bcca0e429716cd68c4da6b7f9ac7976868735937eb63
Successfully built rouge_score


In [ ]:
rouge = evaluate.load('rouge')
results = rouge.compute(predictions=generated_texts,references=reference_texts)
print(results)

{'rouge1': 0.36376045112936156, 'rouge2': 0.15277870256847167, 'rougeL': 0.32989093702531636, 'rougeLsum': 0.32985588469588817}


In [ ]:
bleu = evaluate.load("bleu")
result = bleu.compute(predictions=generated_texts,references=reference_texts,max_order=1)
print(result)

{'bleu': 0.38179239938990034, 'precisions': [0.4215538407576289], 'brevity_penalty': 0.9056788539839462, 'length_ratio': 0.9098597392650274, 'translation_length': 57020, 'reference_length': 62669}


In [ ]:
bleu = evaluate.load("bleu")
result = bleu.compute(predictions=generated_texts,references=reference_texts,max_order=2)
print(result)

{'bleu': 0.23542539798717207, 'precisions': [0.4215538407576289, 0.16028951703966976], 'brevity_penalty': 0.9056788539839462, 'length_ratio': 0.9098597392650274, 'translation_length': 57020, 'reference_length': 62669}
